In [ ]:
from pydantic_ai.messages import ModelMessage
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai import Agent


class PydanticAgent():
    def __init__(
        self,
        name: str,
        model: OpenAIModel, 
        output_type: str,
        system_prompt: str = "You are a helpful assistant.",
    ) -> None:
        """
        Create an agent

        Args:
            agent_id: pydantic_ai.Agent instance
        """
        #self.memory_lst = []
        self.name = name
        self.agent = Agent(model=model, 
                           name=name, 
                           output_type=output_type,
                           system_prompt=system_prompt,
                           model_settings={'tool_choice': 'auto'}
                           )
        self.message_history=list[ModelMessage]

    def ask(self, question: str) -> str:
        result = self.agent.run_sync(
            user_prompt=question,
            #message_history=message_history,
        )


        self.message_history=result.all_messages()

        return result.output
    '''
    def add_event(self, event: str):
        """Add an new event in the memory

        Args:
            event (str): string that describe the event.
        """
        self.message_history.append({"role": "user", "content": f"{event}"})

    def _add_memory(self, memory: str):
        """Monologue in the memory

        Args:
            memory (str): string that generated by the model in the last round.
        """
        self.message_history.append({"role": "assistant", "content": f"{memory}"})
        print(f"----- {self.name} memory: \n{self.message_history} -----\n")

    def ask(self, question: str):

        answer = self._query(question)
        #self._add_memory(answer)
        return answer
    '''
    def get_all_messages(self):
        """Get the memory of the agent"""
        return self.message_history


In [5]:
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.models.openai import OpenAIModel

from pydantic_ai import Agent
model = OpenAIModel(
    #'Qwen/Qwen3-8B'
    model_name='Qwen/Qwen2.5-7B-Instruct',
    provider=OpenAIProvider(
        base_url='https://api.siliconflow.cn/v1',
        api_key='sk-xxx'
    ),
)

play_agent = Agent(model=model, 
                   name='test', 
                   output_type=str,
                   system_prompt='',
                   model_settings={'tool_choice': 'auto'}
                   )

In [40]:
from typing import Literal
from pydantic import BaseModel
class DebateEvaluationOut(BaseModel):
    """
    Analyze both sides of the current situation and provide a comprehensive final answer.
    
    The output should indicate which position is more reasonable and integrate insights from both sides.
    """

    Reason: str 
    debate_answer: Literal['Affirmative', 'Negative', 'Neutral']
    Final_Answer: str 


In [ ]:
model = OpenAIModel(
    #'Qwen/Qwen3-8B'
    model_name='Qwen/Qwen2.5-7B-Instruct',
    provider=OpenAIProvider(
        base_url='https://api.siliconflow.cn/v1',
        api_key='sk-xxx'
    ),
)

class Box(BaseModel):
    width: int
    height: int
    depth: int
    units: str
from typing import Union

Output = Union[str, DebateEvaluationOut]

agent=Agent(
    model=model, 
    name='Judge2',
    output_type=Union[str, DebateEvaluationOut],  # Replace with the correct output_type if needed
)


PydanticSchemaGenerationError: Unable to generate pydantic-core schema for [<class 'str'>, <class '__main__.DebateEvaluationOut'>]. Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.10/u/schema-for-unknown-type

In [ ]:
%load_ext autoreload
%autoreload 2

import dspy
# 初始化语言模型
lm = dspy.LM(
    model='openai/Qwen/Qwen2.5-7B-Instruct',
    api_key='sk-xxx',
    api_base='https://api.siliconflow.cn/v1',
)
from config4prompt import judge_prompt_final2
dspy.configure(lm=lm)
signature = dspy.Signature("debate_topic -> supported_side: Literal['Affirmative', 'Negative']", instructions="please carefully analyze the current situation. You should consider both supporting and opposing factors or arguments, and evaluate which side is more convincing.")

classify = dspy.Predict(signature)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


TypeError: make_signature() missing 1 required positional argument: 'signature'

In [ ]:
answer=classify(debate_topic=judge_prompt_final2)
print(answer.supported_side)
context=lm.history[-1] if lm.history else None

Affirmative


In [ ]:
from typing import Optional,Union

class Evaluator:
    def __init__(
        self,
        name: str,
        model: str,
        api_key: str ,
        api_base: str,
        signature: str,
        instructions: Optional[str] = None,
    ) -> None:
        """
        Initialize a classification agent that uses DSPy to classify questions.

        Args:
            model (Agent): The model name or configuration.
            api_key (str): API key for the language model service.
            api_base (Optional[str]): Base URL for the API (e.g., Azure or custom OpenAI endpoint).
        """
        # 初始化语言模型
        self.lm = dspy.LM(
            model=model,
            api_key=api_key,
            api_base=api_base
        )
        self.signature = signature
        self.instructions = instructions
        self.name = name if name is not None else 'Evaluator'
        # 设置全局默认语言模型
        # dspy.settings.configure(lm=self.lm )

    def run(self, **kwargs) -> str:
        """
        Classifies the given question using a DSPy Predict module.

        Args:
            question (str): Input question to be classified.

        Returns:
            str: Classification result.
        """
        try:
            with dspy.context(lm = self.lm): 
                signature = dspy.Signature(signature=self.signature, instructions=self.instructions)
                classify = dspy.Predict(signature)
                print(classify)
                result = classify(**kwargs)
                return result # 假设返回结果字段是 answer
            
        except Exception as e:
            print(f"Error during classification: {e}")
            return "unknown"

signature="debate_text -> supported_side: Literal['Affirmative', 'Negative']"
instructions="please carefully analyze the current situation. You should consider both supporting and opposing factors or arguments, and evaluate which side is more convincing."
evaluator = Evaluator(#model='openai/Qwen/Qwen2.5-32B-Instruct', 
                        name='Judge2',
                        model='openai/Qwen/Qwen2.5-7B-Instruct',
                        api_key='sk-xxx', 
                        api_base='https://api.siliconflow.cn/v1',
                        signature=signature,
                        instructions= instructions)


In [73]:

debate_text='''[[ ## debate_text ## ]]

[[ ## debate_topic ## ]]
仔细分析现在的经济情况

[[ ## affirmative_side_arguing ## ]]
我同意你的分析，并认为你所提出的观点非常具有针对性和深度。你的分析不仅指出了当前经济环境下的一些关键问题，还提出了合理的政策建议。让我们进一步探讨和细化这些观点：

### 1. **经济增长**

**现状分析：**
- **发达国家**：确实，美国等国家在低通胀和高就业率下表现相对稳定，但也不能忽视欧洲等地区面临的下行压力。德国的表现确实为其他欧洲国家提供了一些借鉴。
- **新兴市场与发展中国家**：你提到东南亚国家的经济增长速度较高，这是值得肯定的亮点，但也需要关注这些国家的长期可持续性。

**问题与挑战：**
- 政策支持和行业合作对于技术创新和产业升级确实至关重要。可以进一步探讨政府如何通过税收减免、研发补贴等方式激励企业和研究机构。
- 疫后恢复确实有进展，但建议从更长远的角度考虑稳定经济复苏的因素，例如持续的投资和支持基础设施建设，以及针对不同地区的特殊政策。

### 2. **通胀**

**现状分析：**
- 你强调了一些欠发达国家通胀相对温和，这需要进一步分析这些国家的具体政策和市场因素。同时，西方高通胀确实由多重因素共同作用。

**问题与挑战：**
- 你说得对，不同国家的情况确实复杂。应对对策可以包括分阶段、有针对性的货币政策调整。例如，美国可以通过减少过剩流动性来应对通胀。
- 通过结构性改革和解决问题的瓶颈措施，可以缓解通胀对消费预期的影响。此外，推动市场竞争可以减少通胀压力。

### 3. **就业**

**现状分析：**
- 生态失衡和技能差距确实是当前就业市场面临的重要问题。你提到通过教育和培训体系来缓解青年就业问题是一个很好的切入点。
- 建立创业政策确实可以创造更多就业机会，但这需要更多具体的政策措施来支持创业者的起步和发展。

### 4. **金融市场**

**现状分析：**
- 股市确实存在泡沫迹象，特别是在科技股。你说的监管和引导民众投资的行为非常重要。
- 债务风险上升在部分新兴市场的确是一个重要问题。加强金融监管和风险管理的关键在于提高透明度和建立健全的金融体系。

**问题与挑战：**
- 引导市场预期和行为确实需要监管机构和市场的共同努力。例如，通过信息披露机制和投资者教育来增强市场信息对称性。
- 加强新兴市场的监管能力不仅需要内部改革，还需要国际间的协作和支持。

### 5. **国际贸易**

**现状分析：**
- 保护主义的抬头确实需要警惕，同时也反映出一些国家对经济依赖的担忧。推动区域经济一体化是一个积极的路径，但需要成员国之间的深度合作。

**问题与挑战：**
- 建立灵活的紧急应变机制可以缓解供应链风险。例如，通过跨国合作，建立更加安全的供应链系统。
- 区域经济一体化确实有潜力缓解贸易保护主义的影响，但在执行过程中需要解决实际存在的问题和障碍。可以推动建立双边或多边自贸协定，加强信息共享和合作机制。

### 结论

我认为你的分析非常全面，并且提出了针对性的政策建议。进一步强调以下几点可以帮助我们更精准地应对当前经济形势：

1. **政策的灵活性和针对性**：不同国家和地区的具体情况差异很大，需要根据实际情况调整政策。
2. **结构性改革的持续性**：着眼长远，通过结构性改革提升经济的韧性和可持续性。
3. **国际合作的重要性**：在全球化背景下，国际合作是解决许多经济问题的关键。

综上所述，我完全同意你的分析，并强烈赞同你提出的观点和建议。

[[ ## negative_side_arguing ## ]]
我非常同意你的进一步分析和建议，你提出了许多有针对性的见解和具体的政策措施，使我们的讨论更加深入和全面。以下是我的进一步分析和观点：

### 1. **经济增长**

**政策建议：**
- **长期可持续性**：针对新兴市场和发展中国家，除了强调短期内的增长速度外，更应关注其长期的经济可持续性。政府可以提供更多关于投资基础设施、技术创新和绿色经济的激励措施，以确保经济增长的长期稳定性。
- **政策灵活性**：根据不同国家和地区的情况，灵活调整政策工具和政策重点，如德国成功的做法可以为其他国家提供借鉴。政府可以设立专项基金支持创新和研发，提供税收减免来鼓励企业和研究机构。

### 2. **通胀**

**政策建议：**
- **分阶段调整货币政策**：根据不同的国家和地区，分阶段调整货币政策。例如，对于美国可以通过减少过剩流动性来应对通胀，但需要结合市场实际情况和经济指标进行实时调整。
- **结构性改革**：通过提高市场竞争、降低生产成本和提高产业效率来缓解通胀压力。政府可以推出政策措施，促进产业结构优化和提高各行业效率，例如通过国家支持的中小企业发展计划，提高市场的整体竞争力。

### 3. **就业**

**政策建议：**
- **教育与培训体系**：建立更完善的教育和培训体系，尤其是针对青年就业问题。政府可以提供职业培训、实习机会和技能认证项目，以帮助青年适应“新经济”中对高技能劳动力的需求。
- **创业政策支持**：进一步完善创业支持政策，包括提供资金、咨询和服务支持，以帮助创业者解决起步和发展中的实际问题。例如，通过政府基金、税收优惠和其他金融工具来扶持初创企业和小微企业。

### 4. **金融市场**

**政策建议：**
- **市场预期管理**：政府和监管机构应共同努力，通过透明的信息披露、加强市场教育和监管来引导市场的合理预期。例如，定期发布市场报告和经济数据，增强市场的信息对称性。
- **风险控制机制**：建立更加完善的金融监管体系，包括风险预警系统和应急运作机制。此外，推动金融科技创新，如区块链和人工智能技术的应用，提高金融市场的透明度和效率。

### 5. **国际贸易**

**政策建议：**
- **区域经济一体化**：推动成员国之间的深度合作，解决实际存在的问题和障碍。通过建立双边或多边自贸协定，加强信息共享和合作机制。
- **灵活应变机制**：建立更加灵活的供应链系统，通过跨国合作来确保供应链安全。例如，建立多通道物流系统，减少单一依赖性，提高供应链的韧性和抗冲击能力。

### 综合结论

你的建议和观点非常全面，并且提出了具体的政策措施来应对当前复杂的经济形势。我们可以通过以下几方面来更精准地应对当前的经济问题：
1. **灵活调整政策以适应不同国家和地区具体情况**：确保政策措施的有效性和针对性。
2. **持续的结构性改革**：提升经济的韧性和可持续性，关注长期稳定性问题。
3. **加强国际合作**：在全球化背景下，国际合作对于解决许多经济问题至关重要，特别是在应对贸易保护主义和国际金融风险方面。

总结来说，你的建议不仅提供了具体的措施，还提出了更具操作性和前瞻性的策略，使得我们的经济政策更加全面和有效。

Respond with the corresponding output fields, starting with the field `[[ ## supported_side ## ]]` (must be formatted as a valid Python Literal[\'Affirmative\', \'Negative\']), and then ending with the marker for `[[ ## completed ## ]]`.'''


evaluator.run(debate_text=debate_text)

Predict(StringSignature(debate_text -> supported_side
    instructions='please carefully analyze the current situation. You should consider both supporting and opposing factors or arguments, and evaluate which side is more convincing.'
    debate_text = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Debate Text:', 'desc': '${debate_text}'})
    supported_side = Field(annotation=Literal['Affirmative', 'Negative'] required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Supported Side:', 'desc': '${supported_side}'})
))


Prediction(
    supported_side='Affirmative'
)

## 工具使用

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import os
from pydantic_ai.agent import Agent
from pydantic_ai.common_tools.tavily import tavily_search_tool

api_key = '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'

from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.models.openai import OpenAIModel

from pydantic_ai import Agent
assert api_key is not None

model = OpenAIModel(
    #'Qwen/Qwen3-8B'
    model_name='Qwen/Qwen2.5-7B-Instruct',
    provider=OpenAIProvider(
        base_url='https://api.siliconflow.cn/v1',
        api_key='sk-xxx'
    ),
)

agent = Agent(
    model=model,
    tools=[tavily_search_tool(api_key)],
    system_prompt='Search Tavily for the given query and return the results.',
)

import nest_asyncio
nest_asyncio.apply()

result = agent.run_sync('告诉我传染病相关最新的头条新闻，给我链接。')
print(result.output)


这里有几条关于传染病的最新新闻：

1. 标题: Northwestern Indiana 男子分享与 HIV 相关的故事 - 芝加哥论坛报
   链接: [As HIV research gutted at federal level, Northwest Indiana man shares his HIV story - Chicago Tribune](https://www.chicagotribune.com/2025/06/29/as-hiv-research-gutted-at-federal-level-nwi-man-shares-his-hiv-story/)
   内容总结: 这位名叫 Gregson 的男子讲述了他的 HIV 诊断经历，包括他如何在诊断后积极面对这一挑战，并表达了对于联邦政府减少资金支持 HIV/AIDS 研究的担忧。 

这条新闻详细描述了一个西北伊利诺伊州的男子与 HIV 相关的亲身经历以及他对于目前资金削减给相关研究带来影响的看法。


In [25]:
from pydantic_ai import Agent,  ToolOutput

from typing import TypeVar
from pydantic_ai.messages import ModelMessage, ModelRequest, ModelRequestPart, ModelResponse, TextPart, UserPromptPart
from collections.abc import  Sequence
from pydantic_ai.tools import Tool, ToolFuncEither, AgentDepsT
OutputDataT = TypeVar("OutputDataT")

class PydanticAgent():
    def __init__(
        self,
        name: str,
        model: OpenAIModel, 
        output_type: type[OutputDataT] | ToolOutput[OutputDataT],
        system_prompt: str = "You are a helpful assistant.",
        tools: Sequence[
            Tool[AgentDepsT] | ToolFuncEither[AgentDepsT, ...]
        ] = (),
    ) -> None:
        """
        Create an agent

        """
        #self.memory_lst = []
        self.name = name
        
        # 判断的是历史all_messages()的context上下文内容
        def completion_detector_hook(messages: list[ModelMessage]) -> list[ModelMessage]:  
            """检测完成标记并处理任务结束"""  
            for message in messages:  
                #print (f">>>> message:  {message} <<<<")
                if isinstance(message, ModelResponse):  
                    for part in message.parts:  
                        #print (f">>>> Part:  {part} <<<<")
                        if isinstance(part, TextPart) and part.content.strip() == "[[ ## completed ## ]]":  
                            # 找到完成标记，可以在这里添加结束逻辑  
                            # 例如：抛出自定义异常来停止执行  
                            print("任务已完成.........")
                            raise TaskCompletedException("任务已完成")

            return messages
        
        class TaskCompletedException(Exception):  
            """任务完成异常"""  
            pass

        self.agent = Agent(model=model, 
                           name=name, 
                           output_type=output_type,
                           system_prompt=system_prompt,
                           tools=tools,
                           model_settings={'tool_choice': 'auto'},
                           retries=3,
                           #history_processors=[completion_detector_hook]
                           )
    
        self.message_history: list[ModelMessage] = []
        self.last_answer=''

    def ask(self, question: str, reset_history=True, max_msg:int=10) -> str:
        
        if self.message_history == None or reset_history==True:
            result = self.agent.run_sync(
            user_prompt=question,
        )
        else:
            result = self.agent.run_sync(
            user_prompt=question,
            message_history= self.message_history[-max_msg:] if len(self.message_history) > max_msg else self.message_history
        )

        self.message_history=result.all_messages()
        self.last_answer=result.output
        
        return result.output

    def get_all_messages(self):
        """Get the memory of the agent"""
        return self.message_history

if __name__ == "__main__":
    agent = PydanticAgent(
        name='test',
        model=model,
        tools=[tavily_search_tool(api_key)],
        system_prompt='Search Tavily for the given query and return the results.',
        output_type=str
    )

    import nest_asyncio
    nest_asyncio.apply()

    result = agent.ask('告诉我传染病相关最新的头条新闻，给我链接。')
    print(result)


根据您的要求，这是最近的关于传染病的头条新闻：

1. 标题：As HIV research gutted at federal level, Northwest Indiana man shares his HIV story - [芝加哥论坛报](https://www.chicagotribune.com/2025/06/29/as-hiv-research-gutted-at-federal-level-nwi-man-shares-his-hiv-story/)
2. 标题：我与著名作家发生了性关系。我们的邂逅让我深感不安 - [AOL](https://www.aol.com/had-sex-famous-writer-encounter-122455251.html)
3. 标题：非洲人类与牲畜界面的抗菌素耐药性传播推断的系统评价 - [自然](https://www.nature.com/articles/s44259-025-00126-y)
4. 标题：常规在养猪场使用抗菌素预防相关的多重耐药性风险 - [自然](https://www.nature.com/articles/s44259-025-00130-2)
5. 标题：受僵尸真菌感染的飞虫可能生活在恐龙时代 - [CNN](https://www.cnn.com/2025/06/29/science/zombie-fungi-fossil-science-newsletter-wt) 

您可以点击链接查看详细内容。


In [ ]:
agent = PydanticAgent(
    name='test',
    model=model,
    tools=[tavily_search_tool(api_key)],
    system_prompt='Search Tavily for the given query and return the results.',
    output_type=str
)

import nest_asyncio
nest_asyncio.apply()

result = agent.ask('告诉我传染病相关最新的头条新闻，给我链接。')
print(result)


AttributeError: 'str' object has no attribute 'output'

In [24]:
print(result)

这里有一些关于传染病的最新新闻：

1. "As HIV research gutted at federal level, Northwest Indiana man shares his HIV story" - [Chicago Tribune](https://www.chicagotribune.com/2025/06/29/as-hiv-research-gutted-at-federal-level-nwi-man-shares-his-hiv-story/)
2. "A systematic review of antimicrobial resistance transmission inferences at the human-livestock interface in Africa" - [Nature](https://www.nature.com/articles/s44259-025-00126-y)
3. "Risk of multidrug resistance in Escherichia coli associated with routine antimicrobial prophylaxis on pig farms" - [Nature](https://www.nature.com/articles/s44259-025-00130-2)

这些文章可能会对你有帮助。
